<a href="https://colab.research.google.com/github/zerobbang/study_colab/blob/main/hongong/ch5_3_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블

- 흐름 : GBM -> XGBoost -> LightGBM 
 - 모델 개발 속도가 개선이 되었는가?
 - 모델의 성능이 개선이 되었는가?

 - 참고) TabNet (2019) - 딥러닝 컨셉

## 랜덤 포레스트
- 결정 트리 모델을 여러개 사용하여 학습하고 결과를 합산하여 예측을 한다.

- 데이터 준비

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, 
                                                                      target, 
                                                                      test_size=0.2, 
                                                                      random_state=42)

- 교차 검증 수행

In [5]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state=42)
scores = cross_validate(rf, train_input, train_target,
                        return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [6]:
rf.fit(train_input,train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


- OOB 샘플
  - 랜덤 포레스트는 훈련 세트에서 중복을 허용하여 부트스트랩 샘플을 만들어 결정 트리 훈련
  - 그때 부트스트랩 샘플에 포함되지 않고 남는 샘플을 의미
  - 이 샘플로 부트스트랩 샘플로 훈련한 결정 트리를 평가하는 검증 세트의 역할을 한다.

In [13]:
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state=42)
rf.fit(train_input,train_target)
print(rf.oob_score_)

0.8934000384837406


## 그레이디언트 부스팅 Gradient Boosting

- 처음은 깊이 낮은 결정 트리를 사용하고 점차적으로 깊이를 늘려가면서 오차를 보완하는 방식으로 앙상블 하는 방법
  - 깊이가 얕은 트리 사용
  - 학습률 매개변수로 속도를 조절
  - 속도가 느리다는 단점 존재

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


-- 과대적합이 거의 되지 않았다

- 학습률을 증가시키고 트리의 개수를 늘려도 보자

In [12]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


-- 결정트리 개수를 늘렸어도 과대적합을 잘 잡고 있다. => 그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 강하다.